In [1]:
import julia

julia.install(quiet=True)

In [2]:
import csv
import glob
import json
import os
import time

import numpy as np
import pandas as pd
from julia import Main
from matpower import start_instance
from matpowercaseframes.constants import COLUMNS

from pypglib import PATH_PYPGLIB

In [3]:
Main.eval("""
    cd("../../PowerModels.jl")
    using Pkg
    Pkg.activate(".")
    Pkg.instantiate()
""")
Main.eval("""
    using PowerModels
    using Ipopt
    using JuMP
""")
Main.eval("""
    nlp_solver = JuMP.optimizer_with_attributes(
        Ipopt.Optimizer,
        "tol" => 1e-6,
        "print_level" => 0
    )
""")

  Activating project at `~/Documents/Git/PowerModels.jl`


<PyCall.jlwrap MathOptInterface.OptimizerWithAttributes(Ipopt.Optimizer, Pair{MathOptInterface.AbstractOptimizerAttribute, Any}[MathOptInterface.RawOptimizerAttribute("tol") => 1.0e-6, MathOptInterface.RawOptimizerAttribute("print_level") => 0])>

In [4]:
output_dir = "../bench"
output_dir_opf = os.path.join(output_dir, "opf")

os.makedirs(output_dir_opf, exist_ok=True)

In [5]:
m = start_instance()

In [19]:
def dump_sol_to_json(sol, output_dir, file_name):
    def convert_ndarray(obj):
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        raise TypeError(f"Object of type {type(obj)} is not JSON serializable")

    json_file_path = os.path.join(output_dir, f"{file_name}.json")

    with open(json_file_path, "w") as json_file:
        json.dump(sol, json_file, indent=4, default=convert_ndarray)


def extract_sol(mpc, m):
    # NOTE: too big if pull rundcopf data
    #   m.eval("_r1.raw = rmfield(_r1.raw, 'task');")
    #   m.eval("_r1 = rmfield(_r1, 'om');")
    #   sol = m.pull("_r1")

    sol = {
        "baseMVA": mpc.baseMVA,
        "version": mpc.version,
        "bus": m.eval("_r1.bus;", verbose=False),
        "gen": m.eval("_r1.gen;", verbose=False),
        "branch": m.eval("_r1.branch;", verbose=False),
        "gencost": mpc.gencost,
        "success": m.eval("_r1.success;", verbose=False),
    }
    return sol


def extract_sol_pm(mpc, result_mpc_pm):
    sol = {
        "baseMVA": mpc.baseMVA,
        "version": mpc.version,
        "bus": mpc.bus,
        "gen": mpc.gen,
        "branch": mpc.branch,
        "gencost": mpc.gencost,
        "success": result_mpc_pm,
    }
    sorted_keys = sorted(result_mpc_pm["solution"]["bus"].keys())
    va = np.array([result_mpc_pm["solution"]["bus"][key]["va"] for key in sorted_keys])
    vm = np.array([result_mpc_pm["solution"]["bus"][key]["vm"] for key in sorted_keys])
    sorted_keys = sorted(result_mpc_pm["solution"]["gen"].keys())
    pg = (
        np.array([result_mpc_pm["solution"]["gen"][key]["pg"] for key in sorted_keys])
        * result_mpc_pm["solution"]["baseMVA"]
    )
    qg = (
        np.array([result_mpc_pm["solution"]["gen"][key]["qg"] for key in sorted_keys])
        * result_mpc_pm["solution"]["baseMVA"]
    )
    sol["gen"][:, 1] = pg
    sol["gen"][:, 2] = qg
    sol["bus"][:, 7] = vm
    sol["bus"][:, 8] = va
    return sol


def run_ed(sol, mpopt):
    # TODO: check at least voltages and power constraints
    if sol["success"] == 1:
        sol_ed = m.runpf(sol, mpopt, verbose=False)
        if sol_ed["success"] == 1:
            status = "Success"
            # NOTE: 1 is index for PG
            cost = m.totcost(sol_ed["gencost"], sol_ed["gen"][:, [1]]).sum()
        else:
            status = "Not Converge"
            cost = "--"
    else:
        sol_ed = {}
        status = "Infeasible"
        cost = "inf"  # Infeasible solution
    return sol_ed, status, cost


def write_to_csv(data_dict, csv_file_path, fieldnames):
    with open(csv_file_path, mode="a", newline="") as csv_file:
        writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
        writer.writerow(data_dict)


def write_acopf_matpower_to_csv(
    file_name, status, cost, python_time_perf, csv_file_path, fieldnames
):
    data_dict = {
        "file_name": file_name,
        "matpower-pip_acopf_status": status,
        "matpower-pip_acopf_cost": cost,
        "matpower-pip_acopf_python_time_perf": python_time_perf,
    }
    write_to_csv(data_dict, csv_file_path, fieldnames)


def write_dcopf_matpower_to_csv(
    file_name, status, cost, python_time_perf, csv_file_path, fieldnames
):
    data_dict = {
        "file_name": file_name,
        "matpower-pip_dcopf_status": status,
        "matpower-pip_dcopf_cost": cost,
        "matpower-pip_dcopf_python_time_perf": python_time_perf,
    }
    write_to_csv(data_dict, csv_file_path, fieldnames)


def write_acopf_powermodels_to_csv(
    file_name, status, cost, python_time_perf, csv_file_path, fieldnames
):
    data_dict = {
        "file_name": file_name,
        "powermodels_acopf_status": status,
        "powermodels_acopf_cost": cost,
        "powermodels_acopf_python_time_perf": python_time_perf,
    }
    write_to_csv(data_dict, csv_file_path, fieldnames)


def julia_script_mpc_to_pm_data():
    return """
        function convert_matrix_to_vector_of_reals(value::Any)
            if isa(value, AbstractMatrix)
                return [Vector(row) for row in eachrow(value)]
            else
                return value
            end
        end

        function convert_dict_value_matrix_to_vector_of_reals(data::Dict)
            for (key, value) in data
                data[key] = convert_matrix_to_vector_of_reals(value)
            end
            return data
        end

        function convert_python_to_matlab_data(dict::Dict{Any, Any})
            new_dict = Dict{String, Any}()
            for (key, value) in dict
                new_dict[string(key)] = convert_matrix_to_vector_of_reals(value)
            end
            return new_dict
        end

        matlab_data = convert_python_to_matlab_data(matlab_data)
        pm_data = PowerModels.parse_matpower(matlab_data, validate=true)
    """

In [8]:
# file_name, status, cost, python_time_perf, csv_file_path, fieldnames

('pglib_opf_case4837_goc.m',
 'Infeasible',
 'inf',
 22.11337129200001,
 '../bench/opf/powermodels_acopf_benchmark_results.csv',
 ['file_name',
  'powermodels_acopf_status',
  'powermodels_acopf_cost',
  'powermodels_acopf_python_time_perf'])

In [29]:
# (
#     file_name,
#     mpc['gen'].shape,
#     mpc_pm['mpc.gen'].shape,
#     len(pm_data['gen']),
#     len(result_mpc_pm['solution']['gen']
# )

('pglib_opf_case2746wop_k.m', (514, 10), (514, 10), 514, 431)

In [20]:
N_WORKERS = 2  # multithreading
opf_dir = os.path.join(PATH_PYPGLIB, "opf")  # 198 cases
mpopt = m.mpoption("verbose", 0, "out.all", 0)
m.push("_mpopt", mpopt)

csv_file_path = os.path.join(output_dir_opf, "powermodels_acopf_benchmark_results.csv")
fieldnames = [
    "file_name",
    "powermodels_acopf_status",
    "powermodels_acopf_cost",
    "powermodels_acopf_python_time_perf",
]

# create results and check already run case
if not os.path.isfile(csv_file_path):
    with open(csv_file_path, mode="w", newline="") as csv_file:
        writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
        writer.writeheader()
df_results = pd.read_csv(csv_file_path)
file_names_m = df_results["file_name"].tolist()

file_paths = list(glob.glob(os.path.join(opf_dir, "**", "*.m"), recursive=True))
for file_path in file_paths:
    # # NOTE: this part for debug only
    # case9_path = os.path.join(PATH_MATPOWER, 'data', 'case9.m')
    # file_path = case9_path

    file_name = os.path.basename(file_path)
    if file_name in file_names_m:
        continue
        # return

    mpc = m.loadcase(file_path)
    # try:
    start_time_perf = time.perf_counter()
    mpc_pm = {("mpc." + key, value) for (key, value) in mpc.items()}
    Main.matlab_data = mpc_pm
    pm_data = Main.eval(julia_script_mpc_to_pm_data())
    result_mpc_pm = Main.eval("result = solve_opf(pm_data, ACPPowerModel, nlp_solver)")

    sol = extract_sol_pm(mpc, result_mpc_pm)
    # sol = extract_sol(mpc, m)  # matpower
    python_time_perf = time.perf_counter() - start_time_perf

    sol_ed, status, cost = run_ed(sol, mpopt)
    # except Oct2PyError as e:
    #     print(e)
    #     print(file_name)
    #     sol = {}
    #     status = "Error"
    #     cost = "Err"
    #     python_time_perf = np.inf

    # NOTE: json is too big, 5 MB for single case
    # dump_sol_to_json(sol, output_dir_opf, file_name)  # too big

    try:
        # NOTE: refactored form using cf to make it faster
        base_name = os.path.splitext(os.path.basename(file_name))[0]
        n_cols = sol["gen"].shape[1]
        columns = COLUMNS.get("gen", list(range(n_cols)))
        columns = columns[:n_cols]
        gen_csv_path = os.path.join(
            output_dir_opf, f"{base_name}_powermodels_acopf.gen.csv"
        )
        with open(gen_csv_path, mode="w", newline="") as file:
            writer = csv.writer(file)
            writer.writerow(columns)  # Write the header
            writer.writerows(sol["gen"])  # Write the data rows

    except (AttributeError, KeyError):
        # errors
        pass

    write_acopf_powermodels_to_csv(
        file_name, status, cost, python_time_perf, csv_file_path, fieldnames
    )
    break

[warn | PowerModels]: no active generators found at bus 1110, updating to bus type from 2 to 1
[warn | PowerModels]: active generators found at bus 2733, updating to bus type from 1 to 2
[warn | PowerModels]: no active generators found at bus 116, updating to bus type from 2 to 1
[warn | PowerModels]: no active generators found at bus 147, updating to bus type from 2 to 1
[warn | PowerModels]: no active generators found at bus 1779, updating to bus type from 2 to 1
[warn | PowerModels]: no active generators found at bus 958, updating to bus type from 2 to 1
[warn | PowerModels]: no active generators found at bus 809, updating to bus type from 2 to 1
[warn | PowerModels]: no active generators found at bus 2002, updating to bus type from 2 to 1
[warn | PowerModels]: no active generators found at bus 2729, updating to bus type from 2 to 1
[warn | PowerModels]: no active generators found at bus 132, updating to bus type from 2 to 1
[warn | PowerModels]: no active generators found at bus 13

ValueError: could not broadcast input array from shape (431,) into shape (514,)

r 187: Float64[]
[info | PowerModels]: removing 3 cost terms from generator 471: Float64[]
[info | PowerModels]: removing 3 cost terms from generator 493: Float64[]
[info | PowerModels]: removing 1 cost terms from generator 7: [7609.999999999999, 0.0]
[info | PowerModels]: removing 3 cost terms from generator 361: Float64[]
[info | PowerModels]: removing 3 cost terms from generator 261: Float64[]
[info | PowerModels]: removing 3 cost terms from generator 194: Float64[]
[info | PowerModels]: removing 3 cost terms from generator 140: Float64[]
[info | PowerModels]: removing 3 cost terms from generator 486: Float64[]
[info | PowerModels]: removing 3 cost terms from generator 397: Float64[]
[info | PowerModels]: removing 3 cost terms from generator 337: Float64[]
[info | PowerModels]: removing 3 cost terms from generator 107: Float64[]
[info | PowerModels]: removing 3 cost terms from generator 102: Float64[]
[info | PowerModels]: removing 1 cost terms from generator 69: [9884.0, 0.0]
[info

In [6]:
# N_WORKERS = 2  # multithreading
# PG = 1  # index for PG
# opf_dir = os.path.join(PATH_PYPGLIB, "opf")  # 198 cases
# mpopt = m.mpoption("verbose", 0, "out.all", 0)
# m.push("_mpopt", mpopt)

# csv_file_path = os.path.join(output_dir_opf, "dcopf_benchmark_results.csv")
# fieldnames = [
#     "file_name",
#     "matpower-pip_dcopf_status",
#     "matpower-pip_dcopf_cost",
#     "matpower-pip_dcopf_python_time_perf",
# ]

# # create results and check already run case
# if not os.path.isfile(csv_file_path):
#     with open(csv_file_path, mode="w", newline="") as csv_file:
#         writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
#         writer.writeheader()
# df_results = pd.read_csv(csv_file_path)
# file_names_m = df_results["file_name"].tolist()

# file_paths = list(glob.glob(os.path.join(opf_dir, "**", "*.m"), recursive=True))
# for file_path in file_paths:
#     # # NOTE: this part for debug only
#     # case9_path = os.path.join(PATH_MATPOWER, 'data', 'case9.m')
#     # file_path = case9_path

#     file_name = os.path.basename(file_path)
#     if file_name in file_names_m:
#         continue
#         # return

#     mpc = m.loadcase(file_path)
#     try:
#         start_time_perf = time.perf_counter()
#         m.push("_mpc", mpc)
#         m.eval("_r1 = rundcopf(_mpc, _mpopt);", verbose=False)
#         sol = extract_sol(mpc, m)
#         python_time_perf = time.perf_counter() - start_time_perf

#         sol_ed, status, cost = run_ed(sol, mpopt)
#     except Oct2PyError as e:
#         print(e)
#         print(file_name)
#         sol = {}
#         status = "Error"
#         cost = "Err"
#         python_time_perf = np.inf

#     # NOTE: json is too big, 5 MB for single case
#     # dump_sol_to_json(sol, output_dir_opf, file_name)  # too big

#     try:
#         # NOTE: refactored form using cf to make it faster
#         base_name = os.path.splitext(os.path.basename(file_name))[0]
#         n_cols = sol["gen"].shape[1]
#         columns = COLUMNS.get("gen", list(range(n_cols)))
#         columns = columns[:n_cols]
#         gen_csv_path = os.path.join(output_dir_opf, f"{base_name}_dcopf.gen.csv")
#         with open(gen_csv_path, mode="w", newline="") as file:
#             writer = csv.writer(file)
#             writer.writerow(columns)  # Write the header
#             writer.writerows(sol["gen"])  # Write the data rows

#     except (AttributeError, KeyError):
#         # errors
#         pass

#     write_dcopf_matpower_to_csv(
#         file_name, status, cost, python_time_perf, csv_file_path, fieldnames
#     )

Octave evaluation error:
error: ij(_,1): out of bound 0 (dimensions are 1x0)
error: called from:
    update_z at line 86, column 21
    convert_x_m2n at line 36, column 20
    network_model_x_soln at line 487, column 47
    network_model_x_soln at line 910, column 16
    network_model_x_soln at line 186, column 16
    network_model_update at line 929, column 13
    run at line 196, column 28
    runpf at line 166, column 5
pglib_opf_case6470_rte.m
Octave evaluation error:
error: ij(_,1): out of bound 0 (dimensions are 1x0)
error: called from:
    update_z at line 86, column 21
    convert_x_m2n at line 36, column 20
    network_model_x_soln at line 487, column 47
    network_model_x_soln at line 910, column 16
    network_model_x_soln at line 186, column 16
    network_model_update at line 929, column 13
    run at line 196, column 28
    runpf at line 166, column 5
pglib_opf_case6515_rte.m
Octave evaluation error:
error: ij(_,1): out of bound 0 (dimensions are 1x0)
error: called from:

In [7]:
os.makedirs(output_dir_opf, exist_ok=True)
df_results = pd.read_csv(csv_file_path)

pd.set_option("display.max_rows", 500)
pd.set_option("display.max_columns", 500)
pd.set_option("display.width", 1000)

print(df_results)

                               file_name matpower-pip_dcopf_status matpower-pip_dcopf_cost  matpower-pip_dcopf_python_time_perf
0               pglib_opf_case4837_goc.m                   Success       893816.6359427541                            15.193566
1              pglib_opf_case2746wop_k.m                   Success      1207752.8238761476                            10.871248
2               pglib_opf_case4601_goc.m                Infeasible                     inf                            15.504618
3               pglib_opf_case4917_goc.m                Infeasible                     inf                            14.942911
4               pglib_opf_case2746wp_k.m                   Success      1627996.7513315084                            10.431077
5               pglib_opf_case197_snem.m                   Success       1.508463783081175                             9.754716
6           pglib_opf_case13659_pegase.m              Not Converge                      --              

In [8]:
# import scipy.io
# import julia
# from julia import Main

# # Initialize Julia and load the required packages
# julia.Julia(compiled_modules=False)
# Main.eval(
# """
# using JuMP, PowerModels, Ipopt
# solver = optimizer_with_attributes(Ipopt.Optimizer, "tol"=>1e-6)
# """
# )

# # List of case files to run (replace with actual .mat file paths or data objects)
# case_files = ["case1.mat", "case2.mat", "case3.mat"]

# # To store the results
# results = []

# # Loop through each case and run the optimization
# for case_file in case_files:
#     # Load the .mat file using scipy.io in Python
#     mat_data = scipy.io.loadmat(case_file)

#     # Pass the .mat data to Julia
#     Main.mat_data = mat_data

#     # Run the AC Optimal Power Flow in Julia and capture the result
#     result = Main.eval('run_ac_opf(mat_data, solver)')

#     # Store the result in the Python results list
#     results.append(result)

# # Now `results` contains the optimization results for each case
# for i, res in enumerate(results):
#     print(f"Result for case {i+1}: {res}")